# Basics of building regression models

Regression models are used to define a model which predicts a continuous variable from a number of observations. Regression modelling is very common in (geotechnical) engineering and is by no means a new thing. Most of the correlations used to infer geotechnical parameters from measurements are indeed regression models.

In machine learning, regression modelling is a common task and several model types exist to define relations between the observations (features) and the outcome (target).

In this notebook, we will show how a simple regression model is created, how it is trained and how the model accuracy is assessed. The demo from the ISFOG pile driving prediction event is used.

## Package imports

A number of Python packages are required. Numpy, Pandas and Plotly are know from the previous tutorial.  scikit-learn is a comprehensive Python package for machine learning which will be used here.

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from plotly import subplots
import plotly.graph_objs as go

## Pile driving data

The dataset is kindly provided by [Cathie Group](http://www.cathiegroup.com).

### Importing data

The first step in any data science exercise is to get familiar with the data. The data is provided in a csv file (```training_data.csv```). We can import the data with Pandas and display the first five rows using the ```head()``` function.

In [ ]:
data = pd.read_csv("Data/training_data_piles.csv")  # Store the contents of the csv file in the variable 'data'
data.head()

The data has 12 columns, containing PCPT data ($ q_c $, $ f_s $ and $ u_2 $), recorded hammer data (blowcount, normalised hammer energy, normalised ENTHRU and total number of blows), pile data (diameter, bottom wall thickness and pile final penetration). A unique ID identifies the location and $ z $ defines the depth below the mudline.

The data has already been resampled to a regular grid with 0.5m grid intervals to facilitate the further data handling.

The hammer energy has been normalised using the same reference energy for all piles in this prediction exercise.

We can see that there is no driving data in the first five rows (NaN values), this is because driving only started after a given self-weight penetration of the pile.

### Summary statistics

We can easily create summary statistics of each column using the ```describe()``` function on the data. This gives us the number of elements, mean, standard deviation, minimum, maximum and percentiles of each column of the data.

We can see that there are more PCPT data points than hammer data points. This makes sense as there is soil data available above the pile self-weight penetration and below the final pile penetration. The pile data is defined in the self-weight penetration part of the profile, so there are slightly more pile data points than hammer record data points.

In [ ]:
data.describe()

### Plotting

We can plot the cone tip resistance, blowcount and normalised ENTHRU energy for all locations to show how the data varies with depth.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=3, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(x=data["qc [MPa]"], y=data["z [m]"], showlegend=False, mode='markers',name='qc')
fig.append_trace(trace1, 1, 1)
trace2 = go.Scatter(x=data["Normalised ENTRHU [-]"], y=data["z [m]"], showlegend=False,
                    mode='markers',name='ENTRHU')
fig.append_trace(trace2, 1, 2)
trace3 = go.Scatter(x=data["Blowcount [Blows/m]"], y=data["z [m]"], showlegend=False,
                    mode='markers',name='Blowcount')
fig.append_trace(trace3, 1, 3)
fig['layout']['xaxis1'].update(title=r'$ q_c \ \text{[MPa]} $', side='top', anchor='y', range=(0, 100), dtick=10)
fig['layout']['xaxis2'].update(title=r'$ \text{Normalised ENTRHU [-]} $', side='top', anchor='y',
                               range=(0, 1), dtick=0.2)
fig['layout']['xaxis3'].update(title=r'$ \text{Observed blowcount [Blows/m]} $', side='top', anchor='y',
                               range=(0, 200), dtick=25)
fig['layout']['yaxis1'].update(title=r'$ z \ \text{[m]} $', range=(40, 0), dtick=2)

fig.show()

The cone resistance data shows that the site mainly consists of sand of varying relative density. In certain profiles, clay is present below 10m. There are also locations with very high cone resistance (>70MPa).

The blowcount profile shows that blowcount is relatively well clustered around a generally increasing trend with depth. The normalised ENTHRU energy is also increasing with depth.

We can isolate the data for a single location by selecting this data from the dataframe with all data. As an example, we can do this for location <i>EK</i>.

In [ ]:
# Select the data where the column 'Location ID' is equal to the location name
location_data = data[data["Location ID"] == "EK"]

We can plot the data for this location on top of the general data cloud.

In [ ]:
trace1_EK = go.Scatter(
    x=location_data["qc [MPa]"], y=location_data["z [m]"], showlegend=False, mode='lines',name='qc EK')
fig.append_trace(trace1_EK, 1, 1)
trace2_EK = go.Scatter(
    x=location_data["Normalised ENTRHU [-]"], y=location_data["z [m]"], showlegend=False,
    mode='lines',name='ENTRHU EK')
fig.append_trace(trace2_EK, 1, 2)
trace3_EK = go.Scatter(
    x=location_data["Blowcount [Blows/m]"], y=location_data["z [m]"], showlegend=False,
    mode='lines',name='Blowcount EK')
fig.append_trace(trace3_EK, 1, 3)
fig.show()

We can see that pile driving started from 5m depth and continued until a depth of 30m, when the pile tip reached a sand layer with $ q_c $ > 60MPa.

Feel free to investigate the soil profile and driving data for the other locations by changing the location ID.

For the purpose of the prediction event, we are interested in the variation of blowcount with $ q_c $, hammer energy, ... We can also generate plots to see the correlations. The data shows significant scatter and non-linear behaviour. We will take this into account for our machine learning model.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=3, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=data["qc [MPa]"], y=data["Blowcount [Blows/m]"], showlegend=False, mode='markers',name='qc')
fig.append_trace(trace1, 1, 1)
trace2 = go.Scatter(x=data["Normalised ENTRHU [-]"], y=data["Blowcount [Blows/m]"], showlegend=False,
                    mode='markers',name='ENTRHU')
fig.append_trace(trace2, 1, 2)
trace3 = go.Scatter(x=data["z [m]"], y=data["Blowcount [Blows/m]"], showlegend=False,
                    mode='markers',name='z')
fig.append_trace(trace3, 1, 3)
fig['layout']['xaxis1'].update(title=r'$ q_c \ \text{[MPa]} $', range=(0, 100), dtick=10)
fig['layout']['xaxis2'].update(title=r'$ \text{Normalised ENTRHU [-]} $', range=(0, 1), dtick=0.2)
fig['layout']['xaxis3'].update(title=r'$ z \ \text{[m]} $', range=(0, 40), dtick=5)
fig['layout']['yaxis1'].update(title=r'$ \text{Observed blowcount [Blows/m]} $', range=(0, 200), dtick=25)

fig.show()

## Basics of machine learning

The goal of the prediction exercise is to define a model relating the input (soil data, hammer energy, pile data) with the output (blowcount).

In ML terminology, we call the inputs (the columns of the dataset except for the blowcount) <i>features</i>. The blowcount is the <i>target variable</i>. Each row in the dataframe represents a <i>sample</i>, a combination of feature values for which the output is known. Data for which a value of the target variable is not yet available is called <i>unseen data</i>.

Before we dive into the code for generating ML models, let's discuss some of the concepts in more detail.

### Machine learning techniques

ML combines several data science techniques under one general denominator. We can discern the following families:

   - Classification: Predict the value of a discrete target variable of a data point based on its features
   - Regression: Predict the value of a continuous target variable based on its features
   - Clustering: Identify groups of similar data points based on their features
   - Dimensionality reduction: Identify the features with the greatest influence on the data
   
The first techniques are examples of <i>supervised learning</i>. We will use data where the output has been observed and use that to <i>train</i> the ML model. Training a model is essentially the optimisation of the coefficients of a mathematical model to minimise the difference between model predictions and observed values. Such a trained algorithm is then capable of making predictions for unseen data.

<img src="Images/machine_learning_concept.png">
<br><center><b>Sketch of the machine learning concept</b></center>

This concept is not fundamentally different from any other type of data-driven modelling. The main advantage of the ML approach is the speed at which the models can be trained and the many types of models available to the engineer.

In our example of pile driving, we have a <b>regression</b> problem where we are training a model to relate features (soil data, hammer energy and pile data) with a continuous target variable (blowcount).

### Model fitting

Machine learning has disadvantages which can lead to problematic situations if the techniques are misused. One of these disadvantages is that the ML algorithm will always find a fit, even if it is a poor one.

The figure below shows an example with data showing a non-linear trend between input and output with some scatter around a trend. We can identify the following situations:

   - Underfitting: If we use a linear model for this data, we are not capturing the trend. The model predictions will be poor;
   - Good fit: If we formulate a model (quadratic in this case) which captures the general trend but allows variations around the trend, we obtain a good fit. In geotechnical problems, we will never get a perfect a fit but if we identify the influence of the input parameters in a consistent manner, we can build good-quality models;
   - Overfitting: If we have a model which perfectly fits all known data points, the prediction for an unseen data point will be poor. The influence of each measurement on the model is too important. The model overfits the data and does not capture the general trends. It just represents the data on which it was trained.

<img src="Images/over_underfitting.png">
<br><center><b>Underfitting and overfitting</b></center>

### Model metrics

To prevent misuse of ML models, we will look at certain model metrics to check the quality. There are several model metrics. Two of the more common ones are the <b>Mean Squared Error (MSE)</b> and the <b>coefficient of determination ($ R^2 $)</b>.

The MSE-value is the normalised sum of quadratic differences. The closer it is to 0, the better the fit.

$$ \text{MSE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2. $$

$ \hat{y}_i $ is the predicted value of the i-th sample and $ y_i $ is the true (measured) value.

The coefficient of determination ($ R^2 $) is a measure for a measure of how well future samples are likely to be predicted by the model. A good model has an $ R^2 $-value which is close to 1.

$$ R^2(y, \hat{y}) = 1 - \frac{\sum_{i=0}^{n_{\text{samples}} - 1} (y_i - \hat{y}_i)^2}{\sum_{i=0}^{n_\text{samples} - 1} (y_i - \bar{y})^2} \quad \text{where} \ \bar{y} =  \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}} - 1} y_i$$

In the example, we will see how we can easily calculate these metrics from the data using the functions available in the ML Python package ```scikit-learn```.

### Model validation

When building a ML model, we will only use a subset of the data for training the model. The other subset is deliberately excluded from the learning process and used to <i>validate</i> the model. The trained model is applied on the unseen data of the validation dataset and the accuracy of the predictions is checked, resulting in a validation score representing the accuracy of the model for the validation dataset.

If our trained model is of good quality, the predictions for the validation dataset will be close to the measured values.

We will partition our data in a training dataset and a validation dataset. For the validation data set, we use seven piles. The other piles will be used as the training dataset.

In [ ]:
validation_ids = ['EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM']
# Training data - ID not in validation_ids
training_data = data[~data['Location ID'].isin(validation_ids)]
# Validation data - ID in validation_ids
validation_data = data[data['Location ID'].isin(validation_ids)]

With these concepts in mind, we can start building up a simple ML model.

## Basic machine learning example: Linear modelling

The most basic type of ML model is a linear model. We are already using linear models in a variety of applications and often fit them without making use of ML techniques. The general equation for a linear model is given below for a model with $ N $ features:

$$ y = a_0 + a_1 \cdot x_1 + a_2 \cdot x_2 + ... + a_N \cdot x_N + \epsilon $$

where $ \epsilon $ is the estimation error.

Based on the training dataset, the value of the coefficients ($ a_0, a_1, ..., a_N $) is determined using optimisation techniques to minimise the difference between measured and predicted values. As the equation shows, a good fit will be obtained when the relation between output and inputs is truly linear. If there are non-linearities in the data, the fit will be less good.

We will illustrate how a linear regression machine learning model is generated from the available driving data.

### Linear model based on normalised ENTHRU only

The simplest linear model depends on only one feature. We can select the normalised energy transmitted to the pile (ENTRHU) as the only feature for illustration purposes.

The mathematical form of the model can be written as:

$$ BLCT = a_o + a_1 \cdot \text{ENTRHU}_{norm} + \epsilon $$

We will create a dataframe $ X $ with only the normalised ENTHRU feature data and we will put the observed values of the target variable (blowcount) in the vector $ y $.

Note that machine learning algorithms will raise errors when NaN values are provided. We need to ensure that we remove such values. We can creata a dataframe ```cleaned_training_data``` which only contains rows with no NaN values.

In [ ]:
features = ['Normalised ENTRHU [-]']
cleaned_training_data = training_data.dropna() # Remove NaN values
X = cleaned_training_data[features]
y = cleaned_training_data["Blowcount [Blows/m]"]

We can now create a linear model. We need to import this type of model from the scikit-learn package. We can fit the linear model to the data using the ```fit()``` method.

In [ ]:
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression().fit(X,y)

At this point, our model has been trained with the data and the coefficients are known. $ a_0 $ is called the intercept and $ a_1 $ to $ a_n $ are stored in ```coef_```. Because we only have one feature, ```coef_``` only returns a single value.

In [ ]:
model_1.coef_, model_1.intercept_

We can plot the data with our trained fit. We can see that the fit follows a general trend but the quality is not great.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False) 

observations_data = go.Scatter(       
    x=X['Normalised ENTRHU [-]'],
    y=y,
    showlegend=True,
    mode='markers',
    name='Data')
fig.append_trace(observations_data, 1, 1)

model_data = go.Scatter(       
    x=np.linspace(0.0, 1, 50),
    y=model_1.intercept_ + model_1.coef_ * np.linspace(0.0, 1, 50),
    showlegend=True,
    mode='lines',
    name='model')
fig.append_trace(model_data, 1, 1)

fig['layout']['xaxis1'].update(title='Normalised ENTRHU [-]') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]')
fig['layout'].update(title='Single feature model')
fig.show()     

We can also calculate the $ R^2 $ score for our training data. The score is below 0.5 and it goes without saying that this model needs improvement.

In [ ]:
model_1.score(X,y)

In the following sections, we will explore ways to improve our model.

### Linearizing features

When using ENTRHU as our model feature, we can see that a linear model is not the most appropriate choice as the relation between blowcount and ENTRHU is clearly non-linear. However, we can <i>linearize</i> features.

For example, we can propose a relation using using a tangent hyperbolic law, which seems to fit better with the data.

In [ ]:
x = np.linspace(0, 1, 100)

fig = subplots.make_subplots(rows=1, cols=1, print_grid=False) 

observations_data = go.Scatter(       
    x=X['Normalised ENTRHU [-]'],
    y=y,
    showlegend=True,
    mode='markers',
    name='Data')
fig.append_trace(observations_data, 1, 1)

tanh_data = go.Scatter(       
    x=x,
    y=80 * np.tanh(5 * x - 0.5),
    showlegend=True,
    mode='lines',
    name='Tanh relation')
fig.append_trace(tanh_data, 1, 1)

fig['layout']['xaxis1'].update(title='Normalised ENTRHU [-]') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]', range=(0, 175))
fig['layout'].update(title='Tanh relation')
fig.show()     

We can create a linearized feature:

$$ (\text{ENTHRU})_{lin} = \tanh(5 \cdot \text{ENTHRU}_{norm} - 0.5) $$

In [ ]:
Xlin = np.tanh(5 * cleaned_training_data[["Normalised ENTRHU [-]"]] - 0.5)

In [ ]:
Xlin

When plotting the linearized data against the blowcount, we can see that a linear relation is more appropriate, although there is still a lot of scatter.

In [ ]:
x = np.linspace(0, 1, 100)

fig = subplots.make_subplots(rows=1, cols=1, print_grid=False) 

observations_data = go.Scatter(       
    x=Xlin['Normalised ENTRHU [-]'],
    y=y,
    showlegend=True,
    mode='markers',
    name='Data')
fig.append_trace(observations_data, 1, 1)

fig['layout']['xaxis1'].update(title='Linearized normalised ENTHRU') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]', range=(0, 175))
fig['layout'].update(title='Tanh relation')
fig.show()     

We can fit another linear model using this linearized feature.

In [ ]:
model_2 = LinearRegression().fit(Xlin, y)

We can check the intercept and the model coefficient:

In [ ]:
model_2.coef_, model_2.intercept_

The model with the linearized feature can then be written as:

$$ BLCT = a_0 + a_1 \cdot (\text{ENTHRU})_{lin} $$ 

We can visualize the fit.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False) 

observations_data = go.Scatter(       
    x=X['Normalised ENTRHU [-]'],
    y=y,
    showlegend=True,
    mode='markers',
    name='Data')
fig.append_trace(observations_data, 1, 1)

model_data = go.Scatter(       
    x=x,
    y=model_2.intercept_ + model_2.coef_ * (np.tanh(5 * x - 0.5)),
    showlegend=True,
    mode='lines',
    name='model')
fig.append_trace(model_data, 1, 1)

fig['layout']['xaxis1'].update(title='Normalised ENTRHU [-]') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]', range=(0, 175))
fig['layout'].update(title='Improved single feature model')
fig.show()     

We can check the $ R^2 $ model score. By linearizing the normalised ENTHRU energy, we have improved our $ R^2 $ score and are thus fitting a model which better describes our data.

In [ ]:
model_2.score(Xlin, y)

### Using engineering knowledge

We know from engineering considerations on the pile driving problem that the soil resistance to driving (SRD) can be expressed as the sum of shaft friction and end bearing resistance. The shaft friction can be expressed as the integral of the unit shaft friction over the pile circumference and length.

If we make the simplifying assumption that there is a proportionality between the cone resistance and the unit shaft friction ($ f_s = \alpha \cdot q_c $), we can write the shaft resistance as follows:

$$ R_s = \int_{0}^{L} \alpha \cdot q_c \cdot \pi \cdot D \cdot dz \approx \alpha \cdot \pi \cdot D \cdot \sum q_{c,i} \cdot \Delta z $$

We can create an additional feature for this. Creating features based on our engineering knowledge will often help us to introduce experience in a machine learning algorithm.

To achieve this, we will create a new dataframe using our training data. We will iteration over all locations in the training data and calculate the $ R_s $ feature using a cumulative sum function. We will then put this data together for all locations.

In [ ]:
enhanced_data = pd.DataFrame() # Create a dataframe for the data enhanced with the shaft friction feature
for location in training_data['Location ID'].unique(): # Loop over all unique locations
    # Select the location-specific data
    locationdata = training_data[training_data['Location ID']==location].copy() 
    # Calculate the shaft resistance feature
    locationdata["Rs [kN]"] = \
        (np.pi * locationdata["Diameter [m]"] * locationdata["z [m]"].diff() * locationdata["qc [MPa]"]).cumsum()
    # Combine data for the different locations in 1 dataframe
    enhanced_data = pd.concat([enhanced_data, locationdata]) 

We can plot the data to see that the clustering of our SRD shaft resistance feature vs blowcount is much better than the clustering of $ q_c $ vs blowcount. We can also linearize the relation between shaft resistance and blowcount.

We can propose the following relation:

$$ BLCT = 85 \cdot \tanh \left( \frac{R_s}{1000} - 1 \right) $$

In [ ]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True) 

qc_data = go.Scatter(       
    x=enhanced_data['qc [MPa]'],
    y=enhanced_data['Blowcount [Blows/m]'],
    showlegend=False,
    mode='markers',
    name='qc data')
fig.append_trace(qc_data, 1, 1)

shaftresistance_data = go.Scatter(       
    x=enhanced_data['Rs [kN]'],
    y=enhanced_data['Blowcount [Blows/m]'],
    showlegend=False,
    mode='markers',
    name='Rs data')
fig.append_trace(shaftresistance_data, 1, 2)

x = np.linspace(0.0, 12000, 50)

model_data = go.Scatter(       
    x=x,
    y=85 * (np.tanh(0.001*x-1)),
    showlegend=False,
    mode='lines',
    name='model')
fig.append_trace(model_data, 1, 2)

fig['layout']['xaxis1'].update(title='qc [MPa]') 
fig['layout']['xaxis2'].update(title='Rs [kN]') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]', range=(0, 175))
fig['layout'].update(title='Improved single feature model')
fig.show()     

We then proceed to filter the NaN values from the data and fit a linear model.

In [ ]:
features = ["Rs [kN]"]
X = enhanced_data.dropna()[features]
y = enhanced_data.dropna()["Blowcount [Blows/m]"]
Xlin = np.tanh((0.001 * X) - 1)

In [ ]:
model_3 = LinearRegression().fit(Xlin, y)

We can print the coefficients of the linear model and visualise the fit.

In [ ]:
model_3.intercept_, model_3.coef_

In [ ]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False) 

observations_data = go.Scatter(       
    x=X['Rs [kN]'],
    y=y,
    showlegend=True,
    mode='markers',
    name='Data')
fig.append_trace(observations_data, 1, 1)

x = np.linspace(0.0, 12000, 50)

model_data = go.Scatter(       
    x=x,
    y=model_3.intercept_ + model_3.coef_ * (np.tanh(0.001 * x - 1)),
    showlegend=True,
    mode='lines',
    name='model')
fig.append_trace(model_data, 1, 1)

fig['layout']['xaxis1'].update(title='Rs [kN]') 
fig['layout']['yaxis1'].update(title='Blowcount [Blows/m]', range=(0, 175))
fig['layout'].update(title='Engineered feature model')
fig.show()     

The fit looks reasonable and this is also reflected in the $ R^2 $ score which is just greater than 0.6. We have shown that using engineering knowledge can greatly improve model quality.

In [ ]:
model_3.score(Xlin, y)

### Using multiple features

The power of machine learning algorithms is that you can experiment with adding multiple features. Adding a feature can improve you model if it has a meaningful relation with the output.

We can use our linearized relation with normalised ENTHRU, shaft resistance and we can also linearize the variation of blowcount with depth:

$$ BLCT = 100 \cdot \tanh \left( \frac{z}{10} - 0.5 \right) $$

Our model with the combined features will take the following mathematical form:

$$ BLCT = a_0 + a_1 \cdot \tanh \left( 5 \cdot \text{ENTHRU}_{norm} - 0.5 \right) + a_2 \cdot \tanh \left( \frac{R_s}{1000} - 1 \right) + a_3 \cdot \tanh \left( \frac{z}{10} - 0.5 \right) $$

We can create the necessary features in our dataframe:

In [ ]:
enhanced_data["linearized ENTHRU"] = np.tanh(5 * enhanced_data["Normalised ENTRHU [-]"] - 0.5)
enhanced_data["linearized Rs"] = np.tanh(0.001 * enhanced_data["Rs [kN]"] - 1)
enhanced_data["linearized z"] = np.tanh(0.1 * enhanced_data["z [m]"] - 0.5)
linearized_features = ["linearized ENTHRU", "linearized Rs", "linearized z"]

We can now fit a linear model with three features. The matrix $ X $ is now an $ n \times 3 $ matrix ($ n $ samples and 3 features).

In [ ]:
X = enhanced_data.dropna()[linearized_features]
y = enhanced_data.dropna()["Blowcount [Blows/m]"]
model_4 = LinearRegression().fit(X,y)

We can calculate the $ R^2 $ score. The score is slightly better compared to our previous model. Given the scatter in the data, this score is already a reasonable value.

In [ ]:
model_4.score(X, y)

### Model predictions

The linear regression model always allows us to write down the mathematical form of the model. We can do so here by filling in the intercept ($ a_0 $) a coefficients $ a_1 $, $ a_2 $ and $ a_3 $ in the equation above.

In [ ]:
model_4.intercept_, model_4.coef_

However, we don't need to explicitly write down the mathematical shape of the model to use it in the code. We can make predictions using the fitted model straightaway.

In [ ]:
predictions = model_4.predict(X)
predictions

We can plot these predictions together with the data. We can see that the model follows the general trend of the data fairly well. There is still significant scatter around the trend.

In [ ]:
fig = subplots.make_subplots(rows=1, cols=3, print_grid=False, shared_yaxes=True)
# Observations
trace1 = go.Scatter(
    x=enhanced_data["Rs [kN]"], y=enhanced_data["Blowcount [Blows/m]"],
    showlegend=True, mode='markers',name='Rs observed')
fig.append_trace(trace1, 1, 1)
trace2 = go.Scatter(
    x=enhanced_data["Normalised ENTRHU [-]"], y=enhanced_data["Blowcount [Blows/m]"], showlegend=True,
                    mode='markers',name='ENTRHU observed')
fig.append_trace(trace2, 1, 2)
trace3 = go.Scatter(x=enhanced_data["z [m]"], y=enhanced_data["Blowcount [Blows/m]"], showlegend=True,
                    mode='markers',name='z observed')
fig.append_trace(trace3, 1, 3)

# Predictions
trace1_pred = go.Scatter(
    x=enhanced_data.dropna()["Rs [kN]"], y=predictions,
    showlegend=True, mode='markers',name='Rs predicted')
fig.append_trace(trace1_pred, 1, 1)
trace2_pred = go.Scatter(
    x=enhanced_data.dropna()["Normalised ENTRHU [-]"], y=predictions, showlegend=True,
                    mode='markers',name='ENTRHU predicted')
fig.append_trace(trace2_pred, 1, 2)
trace3_pred = go.Scatter(
    x=enhanced_data.dropna()["z [m]"], y=predictions, showlegend=True,
                    mode='markers',name='z predicted')
fig.append_trace(trace3_pred, 1, 3)

fig['layout']['xaxis1'].update(title='Rs [kN]', range=(0, 12e3))
fig['layout']['xaxis2'].update(title='Normalised ENTRHU [-]', range=(0, 1), dtick=0.2)
fig['layout']['xaxis3'].update(title='z [m]', range=(0, 40), dtick=5)
fig['layout']['yaxis1'].update(title='Observed blowcount [Blows/m]', range=(0, 200), dtick=25)

fig.show()

During the prediction event, the goal is to fit a machine learning model which further refines the model developed above.

### Model validation

At the start of the exercise, we excluded a couple of locations from the fitting to check how well the model would perform for these unseen locations.

We can now perform this validation exercise by calculating the shaft resistance and linearizing the model features. We can then make predictions with our model developed above.

We will illustrate this for location CB.

In [ ]:
# Create a copy of the dataframe with location-specific data
validation_data_CB = validation_data[validation_data["Location ID"] == "CB"].copy()

In [ ]:
# Calculate the shaft resistance feature and put it in the column 'Rs [kN]'
validation_data_CB["Rs [kN]"] = \
    (np.pi * validation_data_CB["Diameter [m]"] * \
     validation_data_CB["z [m]"].diff() * validation_data_CB["qc [MPa]"]).cumsum()

In [ ]:
# Calculate linearized ENTHRU, Rs and z
validation_data_CB["linearized ENTHRU"] = np.tanh(5 * validation_data_CB["Normalised ENTRHU [-]"] - 0.5)
validation_data_CB["linearized Rs"] = np.tanh(0.001 * validation_data_CB["Rs [kN]"] - 1)
validation_data_CB["linearized z"] = np.tanh(0.1 * validation_data_CB["z [m]"] - 0.5)

In [ ]:
# Create the matrix with n samples and 3 features
X_validation = validation_data_CB.dropna()[linearized_features]
# Create the vector with n observations of blowcount
y_validation = validation_data_CB.dropna()["Blowcount [Blows/m]"]

Given our fitted model, we can now calculate the $ R^2 $ score for our validation data. The score is relatively high and we can conclude that the model generalises well. If this validation score would be low, we would have to re-evaluate our feature selection.

In [ ]:
# Calculate the R2 score for the validation data
model_4.score(X_validation, y_validation)

We can calculate the predicted blowcounts for our validation data.

In [ ]:
validation_predictions = model_4.predict(X_validation)

The predictions (red dots) can be plotted against the actual observed blowcounts. The cone resistance and normalised ENTRHU are also plotted for information.

The predictions are reasonable and follow the general trend fairly well. In the layer with lower cone resistance below (10-15m depth), there is an overprediction of blowcount. This is due to the relatively limited amount of datapoints with low cone resistance in the training data. Further model refinement could address this issue. 

In [ ]:
fig = subplots.make_subplots(rows=1, cols=3, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=validation_data_CB["qc [MPa]"],
    y=validation_data_CB["z [m]"], showlegend=False, mode='lines',name='qc')
fig.append_trace(trace1, 1, 1)
trace2 = go.Scatter(
    x=validation_data_CB["Normalised ENTRHU [-]"],
    y=validation_data_CB["z [m]"], showlegend=False, mode='lines',name='ENTRHU')
fig.append_trace(trace2, 1, 2)
trace3 = go.Scatter(
    x=validation_data_CB["Blowcount [Blows/m]"],
    y=validation_data_CB["z [m]"], showlegend=False, mode='lines',name='Blowcount')
fig.append_trace(trace3, 1, 3)

trace_predictions = go.Scatter(
    x=validation_predictions,
    y=validation_data_CB.dropna()["z [m]"], showlegend=False, mode='markers',name='Blowcount predictions')
fig.append_trace(trace_predictions, 1, 3)

fig['layout']['xaxis1'].update(title=r'$ q_c \ \text{[MPa]} $', side='top', anchor='y', range=(0, 100), dtick=10)
fig['layout']['xaxis2'].update(title=r'$ \text{Normalised ENTRHU [-]} $', side='top', anchor='y',
                               range=(0, 1), dtick=0.2)
fig['layout']['xaxis3'].update(title=r'$ \text{Observed blowcount [Blows/m]} $', side='top', anchor='y',
                               range=(0, 200), dtick=25)
fig['layout']['yaxis1'].update(title=r'$ z \ \text{[m]} $', range=(40, 0), dtick=2)

fig.show()

The process of validation can be automated. The [scikit-learn documentation](https://scikit-learn.org/stable/modules/cross_validation.html) has further details on this.

## Conclusions

This tutorial shows how a basic machine learning model can be built up. The workflow shows the importance of integrating engineering knowledge and to ensure that the models make physical sense.

A machine learning workflow is not fundamentally different from a conventional workflow for fitting a semi-empirical model. The methods available in scikit-learn make the process scaleable to large datasets with only a few lines of code.

The workflow will always consist of the following steps:

   - Select features for the machine learning. Use engineering knowledge to construct features which have a better correlation with the target variable under consideration;
   - Split the dataset in a training dataset and a validation dataset;
   - Select the type of machine learning model you want to use (e.g. Linear regression, Support Vector Machines, Neural Nets, ...);
   - Train the model using the training data;
   - Validate the model on the validation data;

If the validation is successful, the model can be used for predictions on unseen data.

## Practice

We will continue with regression

## Further reading

The internet has a large amount of information available on machine learning. Here are a couple of suggestions to guide your reading:

   - [scikit-learn documentation](https://scikit-learn.org/stable/tutorial/basic/tutorial.html): The scikit-learn package has extensive documentation and provides good general-purpose tutorials;
   - [10 minutes to Pandas](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html): The Pandas package is used extensively in this tutorial to facilitate the data processing. Getting to grips with Pandas is a good idea for every aspiring data enthousiast. The Pandas documentation is extensive and this guide provides you with a good overview of the capabilities of the package;
   - [Towards data science](https://towardsdatascience.com/): A blog with frequent posts on data science topics with contributions for the novice and advanced data scientist. 